+-----------------+--------------+
| Area names      | Acronym      |
+=================+==============+
| Sos Cuzzos      | SC           |
+-----------------+--------------+
| Biarzu          | BZ           |
+-----------------+--------------+
| Pardulette      | PL           |
+-----------------+--------------+
| Tiriedu         | TR           |
+-----------------+--------------+
| Santa Cristina  | ST           |
+-----------------+--------------+
| Funtana Sassa   | FS           |
+-----------------+--------------+
| Monte Scuccurau | MS           |
+-----------------+--------------+
| Pont'ezzu       | PE           |
+-----------------+--------------+
| Iscala Erveghe  | IE           |
+-----------------+--------------+
| Magrina e Figu  | MF           |
+-----------------+--------------+

+------------------------------+--------------------+--------------------+
| Product                      | Application        | Acronym            |
+==============================+====================+====================+
| Kalex                        | Leaf               | KAL                |
+------------------------------+--------------------+--------------------+
| Kalex EVO                    | Leaf               | KEL                |
+------------------------------+--------------------+--------------------+
| Aliette                      | Leaf               | ALL                |
+------------------------------+--------------------+--------------------+
| Kalex                        | Endotherapy        | KAE                |
+------------------------------+--------------------+--------------------+
| Kalex EVO                    | Endotherapy        | KEE                |
+------------------------------+--------------------+--------------------+
| Ridomil                      | Soil               | RIS                |
+------------------------------+--------------------+--------------------+
| Basalt flour + shield effect | Soil               | BSS                |
+------------------------------+--------------------+--------------------+
| Control                      | \-                 | CTR                |
+------------------------------+--------------------+--------------------+

In [ ]:
# Load the necessary packages
library(sf)
library(terra)
library(progress)
library(dplyr)
library(tidyr)
library(lubridate)

### **Crop the RGB + NIR raster files**

In this section the all the Planet Multispectral images (from 01/2018 to 08/2023) inside the foleder are clipped according to the extent of the crowns polygons located in the areas:

In [ ]:
# Set the images folder
images_folder <- "G:/Altri computer/Il_mio_computer/DOTTORATO/PROGETTI/OLIVASTRO_PAULILATINO/Immagini_Planet/MERGE"

# Ottieni la lista dei file nella cartella
file_list <- list.files(images_folder)

# Load the NDVI crowns file
crowns <- st_read("G:/Altri computer/Il_mio_computer/DOTTORATO/PROGETTI/OLIVASTRO_PAULILATINO/QGIS/CHIOME_ALBER_ADS/crowns_class.shp")

write_xlsx(list(sf_object = crowns), "G:/Altri computer/Il_mio_computer/DOTTORATO/PROGETTI/OLIVASTRO_PAULILATINO/QGIS/CHIOME_ALBER_ADS/crowns_class.xlxs")

# Set the output folder
output_folder <- "./ELABORATION/RASTER/IMAGES_CLIPPED"

# Find unique area names
unique_aree <- unique(crowns$AREA)

# Inizializza la barra di avanzamento
progress_bar <- progress::progress_bar$new(
  format = "[:bar] :current/:total (:percent)",
  total = length(file_list) * length(unique_aree)
)
  
# Repeat across the areas
for (area_nome in unique_aree) {
  area_poligoni <- crowns[crowns$AREA == area_nome, ]
  
  # Creates a total extension for polygons in the area
  area_extent <- st_bbox(area_poligoni)
  
  # List all image files in the images folder
  image_files <- list.files(path = images_folder, full.names = TRUE)
  
  # Loop through each image file
for (image_file in image_files) {
  # Skip non-raster files
  if (!grepl("\\.(tif|tiff|jpg|jpeg|png)$", tolower(image_file))) {
    cat("Skipping non-raster file:", image_file, "\n")
    next
  }

  # Read the image raster
  tryCatch({
    image_raster <- terra::rast(image_file)

    # Crop the raster with the full extent of the area
    raster_ritagliato <- terra::crop(image_raster, area_extent)

    # Create complete path to save
    nome_file <- file.path(output_folder, paste(area_nome, "_", tools::file_path_sans_ext(basename(image_file)), ".tif", sep = ""))

    # Save the cropped raster with the name of the area
    terra::writeRaster(raster_ritagliato, nome_file, overwrite = TRUE)

    cat("Raster cut out and saved for the area:", area_nome, "\n")
  }, error = function(e) {
    cat("Error processing file:", image_file, "\n")
    cat("Error message:", conditionMessage(e), "\n")
  })
  progress_bar$tick()
}
}

# Close the progress bar
progress_bar$terminate()

In [ ]:
# Clean the R enviroment
rm(list=ls())

------------------------------------------------------------------------

# Calculate the Vegetation Indices

In this section a list of 20 Vegetation indices is calculate for each date and each area:

In [ ]:
# Set the files directory
directory_path <- "./ELABORATION/RASTER/IMAGES_CLIPPED"

# List the file inside the directory folder
image_files <- list.files(directory_path, pattern = ".tif", full.names = TRUE)

# 
# Imposta la nuova directory per gli indici vegetazionali
output_directory <- "./ELABORATION/RASTER/VI"

# Crea la nuova directory se non esiste
if (!dir.exists(output_directory)) {
  dir.create(output_directory)
}

# Crea un vettore di nomi degli indici
index_names <- c("BLUE", "GREEN","RED", "NIR", "DVI", "EVI", "GCI", "GDVI", 
                 "GEMI", "GNDVI", "GOSAVI", "GRVI", "GSAVI", "IPVI", "MSAVI2",
                 "MSR", "NDVI", "NDWI", "NLI", "OSAVI", "RDVI", "SAVI", "SR",
                 "VARI")

# Inizializza la barra di avanzamento
progress_bar <- progress::progress_bar$new(
  format = "[:bar] :current/:total (:percent)",
  total = length(image_files) * length(index_names)
)

# Funzione per calcolare e salvare un indice vegetazionale specifico
calculate_and_save_index <- function(image_path, output_directory) {
  # Carica l'immagine
  raster_data <- rast(image_path)
  
  # Seleziona le singole bande
  BLUE <- raster_data[[1]]
  GREEN <- raster_data[[2]]
  RED <- raster_data[[3]]
  NIR <- raster_data[[4]]
  
  # Calcola gli indici vegetazionali desiderati
  DVI <- raster_data[[4]] - raster_data[[3]]
  
  EVI <- 2.5 * (raster_data[[4]] - raster_data[[3]]) / 
    (raster_data[[4]] + 6 * raster_data[[3]] - 7.5 * raster_data[[1]] + 1 )
  
  GCI <- (raster_data[[4]] / raster_data[[2]]) - 1 
  
  GDVI <- raster_data[[4]] - raster_data[[2]]
  
  eta = (2 * (raster_data[[4]]^2 - raster_data[[3]]^2) + 1.5 * raster_data[[4]] + 0.5 * raster_data[[3]])/(raster_data[[4]] + raster_data[[3]] + 0.5)
  
  GEMI = eta * (1 - 0.25 * eta) - ((raster_data[[3]] - 0.125)/(1 - raster_data[[3]]))
  GNDVI <- (raster_data[[4]] - raster_data[[2]]) / (raster_data[[4]] + raster_data[[2]])
  
  GOSAVI <- (raster_data[[4]] - raster_data[[2]]) / (raster_data[[4]] + raster_data[[2]] + 0.16)
  
  GRVI <- raster_data[[4]] / raster_data[[2]]
  
  GSAVI <- 1.5 * ((raster_data[[4]] - raster_data[[2]]) / (raster_data[[4]] + raster_data[[2]] + 0.5))

  IPVI <- raster_data[[4]] / (raster_data[[4]] + raster_data[[3]])
  
  MSAVI2 <- (2 * raster_data[[4]] + 1 - sqrt((2 * raster_data[[4]] +1)^2 - 8 * (raster_data[[4]] - raster_data[[3]]))) / 2
  
  MSR <- ((raster_data[[4]]/raster_data[[3]]) - 1) / ((sqrt(raster_data[[4]] / raster_data[[3]])) + 1)
  
  NDVI <- (raster_data[[4]] - raster_data[[3]]) / (raster_data[[4]] + raster_data[[3]])
  
  NDWI <- (raster_data[[2]] - raster_data[[4]]) / (raster_data[[2]] + raster_data[[4]])
  
  NLI <- ((raster_data[[4]])^2 - raster_data[[3]]) / ((raster_data[[4]])^2 + raster_data[[3]])
  
  OSAVI <- (raster_data[[4]] - raster_data[[3]]) / (raster_data[[4]] + raster_data[[3]] + 0.16)
  
  RDVI <- (raster_data[[4]] - raster_data[[3]]) / (sqrt(raster_data[[4]] + raster_data[[3]]))
  
  SAVI <- (1.5 * (raster_data[[4]] - raster_data[[3]])) / (raster_data[[4]] + raster_data[[3]] + 0.5)
  
  SR <- raster_data[[4]] / raster_data[[3]]
  
  VARI <- (raster_data[[2]] - raster_data[[3]]) / (raster_data[[2]] + raster_data[[3]] - raster_data[[1]])

  # Ciclo attraverso gli indici e salva i risultati nella nuova directory
for (index_name in index_names) {
  index_value <- eval(parse(text = index_name))
  terra::writeRaster(index_value, file.path(output_directory, paste0(index_name, "_", basename(image_path))), overwrite = TRUE)
 
  progress_bar$tick()
  
  }

}

# Ciclo attraverso ogni immagine e calcola/salva gli indici
for (image_file in image_files) {
  calculate_and_save_index(image_file, output_directory)
}

# Close the progress bar
progress_bar$terminate()

This is the list of Vegetation Indices used:

+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Spectral Vegetation Index                      | Acr      | Equation                                                            | Ref      |
+================================================+==========+=====================================================================+==========+
| Difference Vegetation Index                    | DVI      | $$                                                                  |          |
|                                                |          | NIR - R                                                             |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Enhanced Vegetation Indexx                     | EVI      | $$                                                                  |          |
|                                                |          | 2.5 + \frac{NIR - R)} {(NIR + 6 * R - 7.5 - B + 1 )}                |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Green Chlorophyll Index                        | GCI      | $$                                                                  |          |
|                                                |          | \frac {NIR}{G}-1                                                    |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Green Difference Vegetation Index              | GDVI     | $$                                                                  |          |
|                                                |          | NIR - G                                                             |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Global Environmental Monitoring Index          | GEMI     | $$                                                                  |          |
|                                                |          | eta + (1 - 0.25 * eta) - \frac{R - 0.125}{1-R}                      |          |
|                                                |          | $$                                                                  |          |
|                                                |          |                                                                     |          |
|                                                |          | $$ eta = \frac{2(NIR^2 - R^2) + 1.5 * NIR + 0.5 * R}{NIR + R + 0.5} |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Green Normalized Difference Vegetation Index   | GNDVI    | $$                                                                  |          |
|                                                |          | \frac{NIR - G} {NIR + G}                                            |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Green Optimized Soil Adjusted Vegetation Index | GOSAVI   | $$                                                                  |          |
|                                                |          | \frac{NIR - G} {NIR + G + 0.16}                                     |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Green Ratio Vegetation Index                   | GRVI     | $$                                                                  |          |
|                                                |          | \frac{NIR}{G}                                                       |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Green Soil Adjusted Vegetation Index           | GSAVI    | $$                                                                  |          |
|                                                |          | 1.5 * \frac{NIR - G}{NIR + G + 0.5}                                 |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Infrared Percentage Vegetation Index           | IPVI     | $$                                                                  |          |
|                                                |          | \frac{NIR}{NIR + R}                                                 |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Modified Soil Adjusted Vegetation Index        | MSAVI2   | $$                                                                  |          |
|                                                |          | \frac{2 * NIR + 1 - \sqrt{(2*NIR + 1)^2 - 8(NIR - R)}}{2}           |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Modified Simple Ratio                          | MSR      | $$                                                                  |          |
|                                                |          | \frac{\frac{NIR}{R} - 1 }{(\sqrt{\frac{NIR}{R})} + 1 }              |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Normalized Difference Vegetation Index         | NDVI     | $$                                                                  |          |
|                                                |          | \frac{NIR - R}{NIR + R}                                             |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Normalized Difference Water Index              | NDWI     | $$                                                                  |          |
|                                                |          | \frac{G - NIR}{G + NIR}                                             |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Non-Linear Index                               | NLI      | $$                                                                  |          |
|                                                |          | \frac{NIR^2 - R}{NIR^2 + R}                                         |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Optimized Soil Adjusted Vegetation Index       | OSAVI    | $$                                                                  |          |
|                                                |          | \frac{NIR - R}{NIR + R + 0.16}                                      |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Renormalized Difference Vegetation Index       | RDVI     | $$                                                                  |          |
|                                                |          | \frac{NIR - R}{\sqrt{NIR + R}}                                      |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Soil Adjusted Vegetation Index                 | SAVI     | $$                                                                  |          |
|                                                |          | \frac{1.5 * (NIR - R)}{(NIR  + R + 0.5)}                            |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Simple Ratio                                   | SR       | $$                                                                  |          |
|                                                |          | \frac{NIR}{R}                                                       |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+
| Visible Atmospherically Resistant Index        | VARI     | $$                                                                  |          |
|                                                |          | \frac{G-R}{G+R-B}                                                   |          |
|                                                |          | $$                                                                  |          |
+------------------------------------------------+----------+---------------------------------------------------------------------+----------+

In [ ]:
library(progress)
library(terra)
library(sf)
library(readr)

# Carica il file CSV con le date
date_info <- read_csv("./CSV/date.csv")

date <- date_info$date

# Carica il file delle chiome NDVI
crowns <- st_read("./SHP/CHIOME_ANALISI_NDVI_2_correct.shp")

# Trova i nomi unici delle aree
unique_aree <- unique(crowns$AREA)

# Definisci le variabili index_vegetations e areas
index_vegetations <- c("DVI", "EVI", "GCI", "GDVI", "GEMI", "GNDVI", "GOSAVI",
                       "GRVI", "GSAVI", "IPVI", "MSAVI2", "MSR", "NDVI", "NDWI",
                       "NLI", "OSAVI","RDVI", "SAVI", "SR", "VARI")

areas <- unique(crowns$AREA)

# Imposta la directory degli indici vegetazionali
vi_directory <- "./RASTER/VI"

# Nuova directory per i raster multilayer
output_directory <- "./RASTER/VI_MERGED"

# Inizializza la barra di progresso
pb <- progress_bar$new(
  format = "[:bar] :percent Elaborazione in corso...",
  total = length(index_vegetations) * length(areas)
)

# Ciclo attraverso gli indici vegetazionali
for (index_vegetation in index_vegetations) {
  # Ciclo attraverso le aree
  for (area in areas) {
    # Trova i file corrispondenti esattamente all'area e all'indice vegetazionale
    matching_files <- list.files(vi_directory, pattern = paste0("^", index_vegetation, "_", area, ".*_MERGED\\.tif$"), full.names = TRUE)

    # Crea un raster multilayer solo con i file corrispondenti
    multilayer <- rast(matching_files)
    
    # Rinomina i layer con l'indice vegetazionale, l'area e la data
    names(multilayer) <- paste0(index_vegetation, "_", area, "_", date)
    
    # Salva il raster multilayer nella nuova directory solo se non esiste già
    output_filename <- file.path(output_directory, paste0(index_vegetation, "_", area, "_MULTILAYER.tif"))
    if (!file.exists(output_filename)) {
      writeRaster(multilayer, filename = output_filename, overwrite = TRUE)
    }
    
    # Aggiorna la barra di progresso
    pb$tick()
  }
}

# Chiudi la barra di progresso
pb$close()

# Stampa un messaggio di completamento
cat("Processo completato con successo.")


Una volta completato il calcolo per ogni area e ogni VI e creato dei multilayer raster per area e VI, devo anadare a selezionare solamente i pixel che stanno dentro le chiome per almeno i 2/3 della loro estensione. Quindi prendo il procedimento utilizzato nei precedenti script e lo applico a questo caso.

### Prepare the crowns file

In [ ]:
# Load all the necessary packages
library(sf)
library(dplyr) 
library(raster)

In [ ]:
# Carica il file delle chiome NDVI
crowns <- st_read("./SHP/CHIOME_ANALISI_NDVI_2_correct.shp")

In [ ]:
VI_directory <- "./RASTER/VI_MERGED"

### Extraxct the NDVI pixels

In this section the pixel values from inside, at least for 2/3 of their extension, of the crowns polygons are extracted. The new files are converted in shp format.

QUI DEVO FARE: PER OGNI FILE PRESENTE NELLA INPUT FOLDER, ESTRAI I VALORI DI VI NELLE CHIOME E CREA DEI NUOVI FILE SHP, UNO PER OGNI FILE.

In [ ]:
library(sf)
library(terra)
library(progress)
library(dplyr)

# Funzione per estrarre il nome dell'area da un file SHP
get_area_name_shp <- function(file_path) {
  # Estrai il nome dell'area tra l'ultimo slash e il punto (ignorando l'estensione .shp)
  area_name_shp <- sub(".*/(.*)\\.shp$", "\\1", file_path)
  return(area_name_shp)
}

# Funzione per estrarre il nome dell'area da un file raster
get_area_name_raster <- function(file_path) {
  # Rimuovi tutto fino al primo underscore incluso
  temp <- gsub("^[^_]*_", "", basename(file_path))
  # Rimuovi "_MULTILAYER" e tutto ciò che segue
  area_name_rast <- gsub("_MULTILAYER.*$", "", temp)
  return(area_name_rast)
}

# Set the path to the shapefile and raster file
shapefile_folder  <- "./SHP/crowns_areas"
raster_folder  <- "./RASTER/VI_MERGED"
output_folder <- "./RASTER/VI_EXTRACTED"

# List files in the folders
raster_files <- list.files(raster_folder, pattern = ".tif$", full.names = TRUE)
shapefile_files <- list.files(shapefile_folder, pattern = ".shp$", full.names = TRUE)

# Estrai i nomi delle aree dai nomi dei file shapefile e raster
shapefile_area_names <- sapply(shapefile_files, get_area_name_shp)
raster_area_names <- sapply(raster_files, get_area_name_raster)

# Inizializza la barra di progresso
pb <- progress_bar$new(
  format = "[:bar] :current/:total (:percent)",
  total = length(raster_files)
)

# Iterate through raster files
for (raster_index in 1:length(raster_files)) {
  # Iterate through shapefiles for each raster
  for (shapefile_index in 1:length(shapefile_files)) {
    # Estrai il nome dell'area dai file shapefile e raster
    shapefile_area_name <- shapefile_area_names[shapefile_index]
    raster_area_name <- raster_area_names[raster_index]
    
    # Verifica se il nome dell'area è lo stesso nei file shapefile e raster
    if (shapefile_area_name == raster_area_name) {
      shp0 <- vect(shapefile_files[shapefile_index])
      rast <- terra::rast(raster_files[raster_index])
  
      # Get the name of one raster layer
      filename <- names(rast[[1]])
  
      # Get the vegetation index name from the filename
      VI <- sub("^(.*?)_.*", "\\1", filename)
  
      # Extract raster values
      ex <- terra::extract(rast, shp0, method = "simple", exact = TRUE, xy = TRUE, ID = FALSE)
  
      # Convert shp from spatvector to sf object
      shp <- st_as_sf(shp0, crs = 32632)

  # If you want to convert the area to another unit, you can use the st_transform function
  shp$estensione <- st_area(st_transform(shp, crs = 32632), square = TRUE) # Change new_crs to the desired coordinate system

  # Filter polygons with at least 2/3 area coverage
  ex_filtered <- ex[ex$fraction >= (2/3),]

  # Create an sf object from the filtered data
  ex_sf <- st_as_sf(ex_filtered, coords = c("x", "y"))
  
  # Assign WGS 84 CRS to your sf object
  ex_sf <- st_set_crs(ex_sf, 32632)

  # Remove the fraction column (no longer needed now)
  ex_sf$fraction <- NULL

  # Remove duplicate rows based on all columns
  ex_sf2 <- distinct(ex_sf)

  # Assign the CRS of ex_sf to polygons
  polygons <- st_as_sf(shp, st_crs(ex_sf2))

  # Perform spatial join based on the position of ex_sf and polygons
  sf_join <- st_join(ex_sf2, polygons)

  # Calculate square side length (3 meters)
  side_length <- 3

  # Create squares using st_buffer
  quadrat_sf <- st_buffer(sf_join, side_length / 2, endCapStyle = "SQUARE")

  # Set CRS (EPSG:32632)
  quadrat_sf <- st_set_crs(quadrat_sf, 32632)
  
  # Elimina la colonna estensione
  quadrat_sf$estensione <- NULL 

  # Rinomina manualmente i campi problematici
  field_names <- names(quadrat_sf)[1:68]
  new_field_names <- gsub(".*_", "", field_names)  # Estrae solo l'ultima parte del nome dopo l'underscore
  
  # # Rimuovi i trattini dalle date
  new_field_names <- gsub("-", "", new_field_names)
  
  names(quadrat_sf)[1:68] <- new_field_names
  
   # Generate output filename based on the shapefile name
      area_name <- tools::file_path_sans_ext(basename(shapefile_files[shapefile_index]))
      output_filename <- file.path(output_folder, paste0(area_name, "_", VI, ".gpkg"))
  
      # Scrivi il file shapefile
      st_write(quadrat_sf, output_filename, driver = "GPKG", append = FALSE)
  
      # Aggiorna la barra di progresso
      pb$tick()
    } else {
      # Issue a warning with the problematic file path
      warning("File does not exist or path is NA at raster index:", raster_index, ", shapefile index:", shapefile_index)
      if (is.na(shapefile_files[shapefile_index])) {
        warning("Problematic shapefile path: NA")
      } else if (!file.exists(shapefile_files[shapefile_index])) {
        warning("Problematic shapefile path:", shapefile_files[shapefile_index])
      }
      if (!file.exists(raster_files[raster_index])) {
        warning("Problematic raster path:", raster_files[raster_index])
      }
    }
  }
}

# Chiudi la barra di progresso alla fine del loop
pb$close()

In [ ]:
# Set the input folder of the extracted ndvi files
INPUT_folder <- "./RASTER/VI_EXTRACTED"

# Get the list of SHP files in the folder
INPUT_shp <- list.files(path = INPUT_folder, pattern = "\\.gpkg$", full.names = TRUE)

# Create a progress bar with the total number of shp files
pb <- progress_bar$new(total = length(INPUT_shp), 
                       format = "[:bar] :current/:total (:percent)")

# Load all the shp files in a list of sf objects
shp_list <- lapply(INPUT_shp, function(file) {
  pb$tick()  # Update the progress bar for each iteration
  
  # Extract the raster file name from the full path
  raster_file_name <- tools::file_path_sans_ext(basename(file))
  
  # Read the shapefile and assign the raster file name as an attribute
  shp <- st_read(file)
  shp$raster_file_name <- raster_file_name
  
  return(shp)
})

# Save the list
saveRDS(shp_list, "./LIST/shp.rds")

# Load the list
shp_list2 <- readRDS("./LIST/shp.rds")

# Create a progress bar with a total number of elements in shp_list
pb <- progress_bar$new(total = length(shp_list2), format = "[:bar] :percent :eta")

# Loop for each element of the list
for (i in seq_along(shp_list2)) {
  # Get the numeric columns names that start whith "X"
  numeric_colnames <- grep("^X\\d+", names(shp_list2[[i]]), value = TRUE)
  
  # Remove the prefix "X" from each numeric column
  new_colnames <- gsub("^X", "", numeric_colnames)
  
  # Upload the columns names of the sf data frame
  names(shp_list2[[i]])[names(shp_list2[[i]]) %in% numeric_colnames] <- new_colnames
  
  # Upload the progress bar
  pb$tick()
}

# Save the list
saveRDS(shp_list2, "./LIST/shp2.rds" )


In [ ]:
shp_list3 <- readRDS("./LIST/shp2.rds")

# # Rimuovi le  colonne non necessarie
cols_to_exclude <- c("ID", "raster_file_name")
# 
# Inizializza la barra di avanzamento
pb <- progress_bar$new(total = length(shp_list3), 
                       format = "[:bar] :current/:total (:percent)")

# Define a function to extract the label from raster_file_name
extract_label <- function(raster_file_name) {
  parts <- strsplit(raster_file_name, "_")[[1]]
  if(length(parts) > 1) {
    return(parts[length(parts)])
  } else {
    return(NA)
  }
}

# Define the function with pb as an argument
process_shapefile <- function(shp, pb) {
  label <- as.character(extract_label(shp$raster_file_name[1]))

  result <- shp %>%
    dplyr::select(-one_of(cols_to_exclude)) %>%
    pivot_longer(
      cols = -c(geom, CODICE_, AREA),
      names_to = "date",
      values_to = label
    ) %>%
    mutate(date = as.Date(date, format = "%Y%m%d"))

  pb$tick()

  return(result)
}

# Use lapply with your function
long_format_list <- lapply(shp_list3, function(shp) {
  process_shapefile(shp, pb)
})

# Change the CODICE colum name in COD for each object sf iside the list.
for (i in seq_along(long_format_list)) {
  long_format_list[[i]] <- long_format_list[[i]] %>%
    rename(COD = CODICE_)
}

# Save the long format list
saveRDS(long_format_list, "./LIST/long_format.rds" )

long_format_list <- readRDS("./LIST/long_format.rds")


library(dplyr)

# Definisci una funzione per elaborare ciascun elemento della lista
process_element <- function(element) {
  # Aggiungi una nuova colonna "VI" con i valori dalla quinta colonna
  element$VI <- colnames(element)[5]
  
  # Rinomina la quinta colonna con "VALUE"
  element <- element %>% rename(VALUE = colnames(element)[5])
  
  return(element)
}

# Applica la funzione a ciascun elemento della lista
long_format_list_2 <- lapply(long_format_list, process_element)


# Save the long format list
saveRDS(long_format_list_2, "./LIST/long_format2.rds")

In [ ]:
long_format_list_2 <- readRDS("./LIST/long_format2.rds")

# Conversione della lista sf in un unico data frame
long_format_df <- bind_rows(long_format_list_2)

# Calcolo della media del VI per ogni COD, data e VI
vi_aggregated <- long_format_df %>%
  group_by(COD, date, VI) %>%
  summarise(VALUE = mean(VALUE))

write.csv(vi_aggregated, "./CSV/vi_aggregated.csv" )

Nellla sezione di sopra l'obbiettivo è di creare un dataframe unico comn tutti i valori delle serie temproali di ogni indici vegetazionale e ogni pixel selezionato.

SI DEVE ANCORA SALVARE IL FILE vi_aggregated

In [ ]:
# Set the scipen option to hight value to eliminate the esponential notation of the values
options(scipen = 999)


vi_aggregated <- read_csv("./CSV/vi_aggregated.csv")

In [ ]:
# Esempio con tapply
min_max_values <- tapply(vi_aggregated$VALUE, vi_aggregated$VI, function(x) c(min(x), max(x)))

# Converti i risultati in un dataframe
min_max_df <- data.frame(VI = names(min_max_values), Min = sapply(min_max_values, "[[", 1), Max = sapply(min_max_values, "[[", 2))

# Visualizza il dataframe con i valori minimi e massimi per ogni VI
print(min_max_df)


In [ ]:
# Elimina le righe con VI in c("GEMI", "DVI", "GDVI")
vi_aggregated <- subset(vi_aggregated, !(VI %in% c("GEMI", "DVI", "GDVI")))



In [ ]:
# Crea il box plot
ggplot(vi_aggregated, aes(x = VI, y = VALUE, fill = VI)) +
  geom_boxplot() +
  labs(title = "Box Plot dei Valori per Categoria VI",
       x = "Categoria VI",
       y = "Valore")

In [ ]:
# Creare una directory se non esiste già
dir.create("./CSV/VI_FROM_LIST", showWarnings = FALSE)

# Inizializza la barra di avanzamento
pb <- progress_bar$new(total = length(long_format_list), 
                       format = "[:bar] :current/:total (:percent)")

# Iterare attraverso gli elementi della lista
for (i in seq_along(long_format_list)) {
  # Ottenere il valore dalla colonna AREA e dalla colonna numerica
  area_value <- unique(long_format_list[[i]]$AREA)
  vi_column_name <- names(long_format_list[[i]][, 5])[[1]]

  # Creare un nome univoco per il file CSV
  filename <- paste0("./CSV/VI_FROM_LIST/", area_value, "_", vi_column_name,".csv")
  
  # Salvare l'elemento corrente come file CSV
  write.csv(long_format_list[[i]], file = filename, row.names = FALSE)
  
  pb$tick()

}

In [ ]:
# Set the csv folder directory
csv_folder <- "./CSV/VI_FROM_LIST"

# Get the list of csv files in the folder
INPUT_csv <- list.files(path = csv_folder, pattern = "\\.csv$", full.names = TRUE)

# Inizializza una lista per immagazzinare i dataframe
dataframes_list <- list()

# Itera attraverso i file CSV e leggi ciascun file
for (csv_file in INPUT_csv) {
  # Leggi il file CSV
  current_df <- read_csv(csv_file)
  
  # Aggiungi il dataframe alla lista
  dataframes_list[[length(dataframes_list) + 1]] <- current_df
}

# Combina tutti i dataframe in un unico dataframe
combined_dataframe <- bind_rows(dataframes_list)